In [4]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string

#pd.options.mode.chained_assigment = None

#added to google mount
from google.colab import drive
drive.mount('/content/drive')


#read file
full_df = pd.read_csv("/content/drive/My Drive/Colab_Notebooks/v.csv")
df = full_df[["text"]]
df["text"] = df["text"].astype(str)
full_df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,text
0,President Biden warned President Vladimir V. P...


In [5]:
#add a new column to text as text_lower that only has lower case letters
df["text_lower"] = df["text"].str.lower()
df["texxt"]
df.head()

,text,text_lower
0,President Biden warned President Vladimir V. P...,president biden warned president vladimir v. p...


In [6]:
# drop the new column created in last cell
df.drop(["text_lower"], axis=1, inplace=True)

#remove punctuation to change to space"
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df["text_wo_punct"] = df["text"].apply(lambda text: remove_punctuation(text))

## is wrong with
df.head()


,text,text_wo_punct
0,President Biden warned President Vladimir V. P...,President Biden warned President Vladimir V Pu...


In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

#remove all stop words from text, changes to space
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])


df["text_wo_stop"] = df["text_wo_punct"].apply(lambda text: remove_stopwords(text))
df.head()

,text,text_wo_punct,text_wo_stop
0,President Biden warned President Vladimir V. P...,President Biden warned President Vladimir V Pu...,President Biden warned President Vladimir V Pu...


In [10]:
#remove top 10 frequent words in English
from collections import Counter
cnt = Counter()
for text in df["text_wo_stop"].values:

    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)

[('President', 2),
 ('Biden', 1),
 ('warned', 1),
 ('Vladimir', 1),
 ('V', 1),
 ('Putin', 1),
 ('Russia', 1),
 ('sanctions', 1),
 ('would', 1),
 ('follow', 1)]

In [21]:


FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

df["text_wo_stopfreq"] = df["text_wo_stop"].apply(lambda text: remove_freqwords(text))
df.head()



,text,text_wo_punct,text_wo_stop,text_wo_stopfreq
0,President Biden warned President Vladimir V. P...,President Biden warned President Vladimir V Pu...,President Biden warned President Vladimir V Pu...,withdraw forces engage diplomatic efforts reso...


In [22]:
# Drop text_wo_punct and text_wo_stop which are no more needed 
# df.drop(["text_wo_punct", "text_wo_stop"], axis=1, inplace=True)

#remove rare words in English and add space
n_rare_words = 103
cnt.most_common()[:-n_rare_words-1:-1]



[('crisis', 1),
 ('resolve', 1),
 ('efforts', 1),
 ('diplomatic', 1),
 ('engage', 1),
 ('forces', 1),
 ('withdraw', 1),
 ('follow', 1),
 ('would', 1),
 ('sanctions', 1),
 ('Russia', 1),
 ('Putin', 1),
 ('V', 1),
 ('Vladimir', 1),
 ('warned', 1),
 ('Biden', 1),
 ('President', 2)]

In [23]:
# Drop the two columns which are no more needed 
df.drop(["text_wo_punct", "text_wo_stop"], axis=1, inplace=True)

n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

df["text_wo_stopfreqrare"] = df["text_wo_stopfreq"].apply(lambda text: remove_rarewords(text))
df.head()

,text,text_wo_stopfreq,text_wo_stopfreqrare
0,President Biden warned President Vladimir V. P...,withdraw forces engage diplomatic efforts reso...,


In [24]:
from nltk.stem.porter import PorterStemmer

# Drop the two columns 
df.drop(["text_wo_stopfreq", "text_wo_stopfreqrare"], axis=1, inplace=True) 

stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

df["text_stemmed"] = df["text"].apply(lambda text: stem_words(text))
df.head()

,text,text_stemmed
0,President Biden warned President Vladimir V. P...,presid biden warn presid vladimir V. putin of ...
